In [19]:
from visualizer import get_local
import torch
get_local.activate()

In [20]:
t = torch.ones(2, 3, requires_grad=True, device='cpu')

class c1:
    def __init__(self, value):
        self.value = value

    def f(self, x):
        return torch.ones(2, x, requires_grad=True)
    
    @get_local('result')
    def get_value(self):
        result = self.f(self.value)
        return result
    
@get_local('x2', 'y2', 'z2')
def fun2(v=1):
    y2 = {'k': v *2}
    x2 = [v]
    z2 = (v * 3)
    return x2, y2, z2

In [21]:
c = c1(4)
c.get_value()

fun2(5)
fun2(t)

([tensor([[1., 1., 1.],
          [1., 1., 1.]], requires_grad=True)],
 {'k': tensor([[2., 2., 2.],
          [2., 2., 2.]], grad_fn=<MulBackward0>)},
 tensor([[3., 3., 3.],
         [3., 3., 3.]], grad_fn=<MulBackward0>))

In [22]:
@get_local('closure_var')
def create_closure_test():
    """创建一个包含闭包变量的测试案例"""
    # 闭包变量 (将被嵌套函数引用)
    closure_var = 10
    var = 1
    
    def nested_func():
        """嵌套函数：修改并返回闭包变量"""
        nonlocal closure_var  # 声明为闭包变量
        closure_var += 1
        return closure_var
    
    return nested_func()

def create_plain_test(arg1 = 1):
    var = 1
    li = [1, 2, 3]
    return var, li, arg1

create_closure_test 和 create_plain_test 中变量类型是不同的

In [23]:
fun = create_closure_test
code = fun.__code__

for attr in dir(code):
    if attr.startswith('co_'):
        print(f"{attr}:\t{getattr(code, attr)}")

co_argcount:	0
co_cellvars:	()
co_code:	b'y\x0e\x88\x01|\x00|\x01\x8e\x01}\x02W\x00nT\x04\x00t\x00k\nrb\x01\x00}\x03\x01\x00z6t\x01d\x01|\x03\x83\x02\x01\x00t\x01d\x02\x83\x01\x01\x00t\x01t\x02\xa0\x03\x88\x01j\x04\xa1\x01\x83\x01\x01\x00t\x01\x88\x00\x83\x01\x01\x00t\x05d\x03\x83\x01\x01\x00W\x00d\x00d\x00}\x03~\x03X\x00Y\x00n\x02X\x00|\x02d\x04\x19\x00|\x02d\x03d\x00\x85\x02\x19\x00\x02\x00}\x04}\x05g\x00}\x06x\x90|\x05D\x00]\x88}\x07t\x06|\x07d\x05\x83\x02r\xaa|\x06\xa0\x07|\x07\xa0\x08\xa1\x00\xa0\t\xa1\x00\xa0\n\xa1\x00\xa1\x01\x01\x00q\x84t\x0b|\x07t\x0c\x83\x02s\xc0t\x0b|\x07t\r\x83\x02\x90\x01r\x02g\x00}\x08x0|\x07D\x00](}\t|\x08\xa0\x07t\x06|\td\x05\x83\x02r\xec|\t\xa0\x08\xa1\x00\xa0\t\xa1\x00\xa0\n\xa1\x00n\x02|\t\xa1\x01\x01\x00q\xcaW\x00|\x06\xa0\x07|\x08\xa1\x01\x01\x00q\x84|\x06\xa0\x07|\x07\xa1\x01\x01\x00q\x84W\x00t\x0e\x88\x02\x83\x01j\x0f\x88\x01j\x10\x19\x00\xa0\x07|\x06\xa1\x01\x01\x00|\x04S\x00'
co_consts:	(None, 'UnboundLocalError:', 'Function code And Extra code

In [24]:
get_local.cache

{'c1.get_value': [['result'],
  [array([[1., 1., 1., 1.],
          [1., 1., 1., 1.]], dtype=float32)]],
 'fun2': [['x2', 'y2', 'z2'],
  [[5], {'k': 10}, 15],
  [[array([[1., 1., 1.],
           [1., 1., 1.]], dtype=float32)],
   {'k': tensor([[2., 2., 2.],
            [2., 2., 2.]], grad_fn=<MulBackward0>)},
   array([[3., 3., 3.],
          [3., 3., 3.]], dtype=float32)]],
 'create_closure_test': [['closure_var']]}

In [25]:
get_local.clear()

In [26]:
import dis
dis.dis(c1)

Disassembly of __init__:
  5           0 LOAD_FAST                1 (value)
              2 LOAD_FAST                0 (self)
              4 STORE_ATTR               0 (value)
              6 LOAD_CONST               0 (None)
              8 RETURN_VALUE

Disassembly of f:
  8           0 LOAD_GLOBAL              0 (torch)
              2 LOAD_ATTR                1 (ones)
              4 LOAD_CONST               1 (2)
              6 LOAD_FAST                1 (x)
              8 LOAD_CONST               2 (True)
             10 LOAD_CONST               3 (('requires_grad',))
             12 CALL_FUNCTION_KW         3
             14 RETURN_VALUE

Disassembly of get_value:
 70           0 SETUP_EXCEPT            14 (to 16)

 71           2 LOAD_DEREF               1 (func)
              4 LOAD_FAST                0 (args)
              6 LOAD_FAST                1 (kwargs)
              8 CALL_FUNCTION_EX         1
             10 STORE_FAST               2 (result)
             12 PO